In [1]:
import requests
import base64
import json
import re
import hashlib

import mysql.connector
from dotenv import load_dotenv
import os


In [2]:
def campos(fields):
    cad = ''
    for x in fields.keys():
        cad += x + ' = ' + fields[x] + ', '
    return cad[:-2]

def leeactzl(user, repo_name, path_to_file):
    json_url ='https://api.github.com/repos/{}/{}/contents/{}'.format(user, repo_name,
                                                                      path_to_file)
    response = requests.get(json_url) #get data from json file located at specified URL 

    if response.status_code == requests.codes.ok:
        jsonResponse = response.json()  # the response is a JSON
        #the JSON is encoded in base 64, hence decode it
        content = base64.b64decode(jsonResponse['content'])
        #convert the byte stream to string
        jsonString = content.decode('utf-8')
        return json.loads(jsonString)
 
    else:
        return 'Content was not found.'
    
def calcquerys(finalJson):
    
    querys = []
    for registro in finalJson:
        if registro['firma'] == hashlib.sha1(registro["instruccion"].encode('utf-8')).hexdigest():
            dt_query = json.loads(registro['instruccion'])
            
            quetmp = []
            for y in dt_query.keys():
                if dt_query[y]['op'] == 'UPDATE':
                    query  = str(dt_query[y]['op']) + ' ' + y + ' SET ' + campos(dt_query[y]['fields'])
                    query += ' WHERE '
                elif dt_query[y]['op'] == 'INSERT INTO':
                    query  = query = str(dt_query[y]['op']) + ' ' + y
                    query += ' ' + str(tuple(list(dt_query[y]['fields'].keys())))+ ' VALUES '
                    query +=  str(tuple(list(dt_query[y]['fields'].values())))
                    query  = query.replace("'", "")
                    filtro_update = 'coduser = ' + dt_query[y]['fields']['coduser']
                quetmp.append(query)
                
            quetmp = [q + filtro_update if 'UPDATE' in q else q for q in quetmp]

            querys.extend(quetmp)

            #break # <--== Pilas quitar en produccion

    
    patron = re.compile('[0-9]{4}-[0-9]{2}-[0-9]{2}')

    for i, query in enumerate(querys):
        for r in patron.findall(querys[i]):
            querys[i] = querys[i].replace(r, "'" + r + "'")

    return querys

def conexion(query):
    try:
        cnx = mysql.connector.connect(user=sql_id, password=sql_pw, host='127.0.0.1', database='clientes')
        cursor = cnx.cursor()
        seleccion = (query)
        cursor.execute(seleccion)
        cnx.commit()
        return 'ok'
    except:
        cnx = mysql.connector.connect(user=sql_id, password=sql_pw, host='127.0.0.1', database='clientes')
        cursor = cnx.cursor()
        seleccion = (query)
        cursor.execute(seleccion)
        return list(cursor)

In [3]:
user = 'sistelca'
repo_name = 'desechosSolidos'
path_to_file = 'data_actzl.json'

finalJson = leeactzl(user, repo_name, path_to_file)


In [50]:
# Ojo hay que registrar el hash en db receptora para vefiricar que no este y ejecutar 
# query

load_dotenv()

sql_id = os.getenv("SQL_ID")
sql_pw = os.getenv("SQL_PW")

bischo = calcquerys(finalJson)


for bis in bischo:
# pendiente por probar
     ok = conexion(bis)
     print(bis + ok)


UPDATE datos_red SET fech_pag = '2021-03-21' WHERE coduser = 347
INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc) VALUES (347, '2021-04-05', 100000, '2021-04-21')
UPDATE datos_per SET fech_ing = '2021-04-05', fech_ven = '2021-04-21' WHERE coduser = 347
UPDATE datos_red SET fech_pag = '2021-03-22' WHERE coduser = 473
INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc) VALUES (473, '2021-04-06', 100000, '2021-04-22')
UPDATE datos_per SET fech_ing = '2021-04-06', fech_ven = '2021-04-22' WHERE coduser = 473
UPDATE datos_red SET fech_pag = '2021-04-06' WHERE coduser = 452
INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc) VALUES (452, '2021-04-06', 200000, '2021-05-07')
UPDATE datos_per SET fech_ing = '2021-04-06', fech_ven = '2021-05-07' WHERE coduser = 452
UPDATE datos_red SET fech_pag = '2021-03-23' WHERE coduser = 252
INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc) VALUES (252, '2021-04-07', 100000, '2021-04-23')
UPDATE datos_p

In [7]:
lea = finalJson[0]
lea

{'id': 1,
 'fecha': '2021-05-04 12:53:19',
 'instruccion': '{"datos_red":{"op":"update","set":" fech_pag=\'2021-05-18\' ","filtro":" coduser=\'341\'"},"histori_pags":{"op":"insert into","set":"coduser, fech_pag, cant_bf, fech_venc, anfit, cod","filtro":" (341, \'2021-05-04\', 200000, \'2021-06-18\', \'192.168.45.221\', \'9460bc8a36a908bababe5b07c8b586a6\')"},"datos_per":{"op":"update","set":" fech_ing=\'2021-05-04\', fech_ven=\'2021-06-18\' ","filtro":" coduser=\'341\'"}}',
 'firma': '542a3ff0c859bd59942c10abee05b82b88ed0294'}

In [13]:
json.loads(lea['instruccion'])


{'datos_red': {'op': 'update',
  'set': " fech_pag='2021-05-18' ",
  'filtro': " coduser='341'"},
 'histori_pags': {'op': 'insert into',
  'set': 'coduser, fech_pag, cant_bf, fech_venc, anfit, cod',
  'filtro': " (341, '2021-05-04', 200000, '2021-06-18', '192.168.45.221', '9460bc8a36a908bababe5b07c8b586a6')"},
 'datos_per': {'op': 'update',
  'set': " fech_ing='2021-05-04', fech_ven='2021-06-18' ",
  'filtro': " coduser='341'"}}